In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Souvikcmsa/SentimentAnalysisDistillBERT")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("Souvikcmsa/SentimentAnalysisDistillBERT")

In [6]:
# Input sample sentences
sentence = """no problem i will send you the information shortly. then i can give you a follow call friday around five.
maybe i give you a call back. do you prefer to call around this one after lunch time.
okay thank you so much for your time.
then okay i will call again on tuesday bye bye.
sure I can call you back on next tuesday. thanks so much.
hi this is leon calling from income ntuc. is it a good time to speak for a while?
ya hello good afternoon speak to leon please. afternoon mister leon my name is jimmy and im actually calling from income.
ya you see even just now he message me he said wah i wanna die.
currently we partnership with abcd. can i check with you is this a good time to speak with you right now?
good evening may i speak to mister leon please? my name is Leon and im calling from Income. may I speak with you for a few minutes?
thank you for calling income. this is leon how may i assist you?
ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and
as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty
i dont want to talk with you
I hate you
"""

In [7]:
# Split sentence and store in a list
sentence_ls = sentence.split("\n")
# Remove the top and last empty string
sentence_ls.pop(0)
sentence_ls.pop(-1)

''

In [8]:
import numpy as np
def softmax(x, axis):
    e_x = np.exp(x - np.max(x, axis))
    return e_x / (e_x.sum(axis) + 1e-9)

In [9]:
def batch_tokenizer(sentence_ls):
    output_ls = []
    for sentence in sentence_ls:
        cur_input = tokenizer(sentence, return_tensors = "pt")
        output = model(**cur_input).logits.detach().numpy()[0]
        soft_output = softmax(output, 0)
        output_ls.append(soft_output)
    return output_ls

In [10]:
# Label 0 is negative, Label 1 is neutral, Label 2 is positive
output_ls = batch_tokenizer(sentence_ls)

In [11]:
# generate label
def generate_label(output_ls, sentence_ls):
    new_output_ls = []
    label = ['negative', 'neutral', 'positive']
    for i, output in enumerate(output_ls):
        index = np.argmax(output)
        new_output_ls.append((sentence_ls[i], label[index]))
    return new_output_ls

In [12]:
generate_label(output_ls, sentence_ls)

[('maybe i give you a call back. do you prefer to call around this one after lunch time.',
  'neutral'),
 ('okay thank you so much for your time.', 'positive'),
 ('then okay i will call again on tuesday bye bye.', 'neutral'),
 ('sure I can call you back on next tuesday. thanks so much.', 'positive'),
 ('hi this is leon calling from income ntuc. is it a good time to speak for a while?',
  'neutral'),
 ('ya hello good afternoon speak to leon please. afternoon mister leon my name is jimmy and im actually calling from income.',
  'positive'),
 ('ya you see even just now he message me he said wah i wanna die.',
  'negative'),
 ('currently we partnership with abcd. can i check with you is this a good time to speak with you right now?',
  'positive'),
 ('good evening may i speak to mister leon please? my name is Leon and im calling from Income. may I speak with you for a few minutes?',
  'positive'),
 ('thank you for calling income. this is leon how may i assist you?',
  'positive'),
 ('ya he